<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/ML2024_ex14notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex14notecookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML)


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

### 機械学習ライブラリ scikit-learn のいろいろ

# 混合正規分布(Gaussian Mixture Model)モデル
from sklearn.mixture import GaussianMixture

# ロジスティック回帰モデル
from sklearn.linear_model import LogisticRegression

# SVM（識別器）
from sklearn.svm import SVC

# 階層型ニューラルネット（多層パーセプトロン）
from sklearn.neural_network import MLPClassifier

---
## 密度推定と統計的識別の実験
---

----
### 手書き数字識別やってみる

クラスごとのデータの分布を正規分布と仮定して事後確率を推定する方法で，手書き数字識別やってみましょう．


#### 準備

In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
datL = rv['datL'].astype(float)
labL = rv['labL']
datT = rv['datT'].astype(float)
labT = rv['labT']
print(datL.shape, labL.shape, datT.shape, labT.shape)

K = 10 # クラス数
D = datL.shape[1] # データの次元数 28 x 28 = 784

In [ ]:
# データを画像として表示するための関数
#
def display(data, nx, ny, nrow=28, ncol=28, gap=4):

    assert data.shape[0] == nx*ny
    assert data.shape[1] == nrow*ncol

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = data[iy*nx+ix].reshape((nrow, ncol))

    # 画像の出力
    plt.axis('off')
    plt.imshow(img, cmap = 'gray')
    plt.show()

In [ ]:
nx, ny = 10, 5
display(datL[:50], nx, ny)
for iy in range(ny):
    print(labL[iy*nx:(iy+1)*nx])

In [ ]:
XL = datL
YL = labL
NL, D = datL.shape
K = 10

#### 事前確率を推定

クラスごとの学習データの比率でクラス事前確率 $p(y)$ を推定．

In [ ]:
py = np.empty(K)
for k in range(K):
    py[k] = np.sum(YL==k)/NL
print('### 事前確率')
print('p(y) = ', py)

#### 正規分布の当てはめ

クラスごとのデータの分布 $p(\mathbf{x}|y)$ を推定．
多変量正規分布でモデル化し，それぞれの平均と分散共分散行列を求めます．

In [ ]:
model = np.empty(K, dtype=object)
for k in range(K):
    XX = XL[YL==k, :]
    model[k] = GaussianMixture(n_components=1, covariance_type='full')
    model[k].fit(XX)

クラスごとの正規分布の平均パラメータを可視化してみるとこんなん．

In [ ]:
xm = np.empty((K, D))
for k in range(K):
    xm[k, :] = model[k].means_[0, :]
display(xm, K, 1)

#### 識別させてみる

与えられたデータを事後確率の推定値が最大となるクラスへと分類する方法で，識別率を算出してみます．
クラスごとのデータの平均をプロトタイプとする最短距離法でも同じことをやり，結果を比較します．前者を「方法1」，後者を「方法2」とします．

まずは学習データから．

In [ ]:
X = datL
Y = labL
N, _ = datL.shape

score = np.empty(K)
ncorrect1 = ncorrect2 = 0

for n in range(N):
    if n % 500 == 0:
        print(f'{n}/{N}')
    xx = X[n, :]

    # 方法1: 事後確率の推定値が最大のクラスに識別
    for k in range(K):
        # log(p(y)p(x|y)) = log(p(y)) + log(p(x|y))
        score[k] = np.log(py[k]) + model[k].score(xx[np.newaxis, :])
    if np.argmax(score) == Y[n]:
        ncorrect1 += 1

    # 方法2: 最短距離法で識別
    d2 = np.sum(np.square(xx[np.newaxis, :] - xm), axis=1)
    if np.argmin(d2) == Y[n]:
        ncorrect2 += 1

print(f'方法1の識別率: {ncorrect1}/{N} = {ncorrect1/N:.3f}')
print(f'方法2の識別率: {ncorrect2}/{N} = {ncorrect2/N:.3f}')

ex14notebookA の「よだんだよん」に書いたことですが，最短距離法（方法2）は，方法1で正規分布モデルに強い制約を加えたものとみなすことができます．
学習データの場合，そのような制約がなく $p(\mathbf{x}|y)$ に任意の正規分布を当てはめられる方法1の方が高い識別率を示しています．


では，テストデータではどうでしょう．

In [ ]:
X = datT
Y = labT
N, _ = datT.shape

score = np.empty(K)
ncorrect1 = ncorrect2 = 0

for n in range(N):
    if n % 500 == 0:
        print(f'{n}/{N}')
    xx = X[n, :]

    # 方法1: 事後確率の推定値が最大のクラスに識別
    for k in range(K):
        # log(p(y)p(x|y)) = log(p(y)) + log(p(x|y))
        score[k] = np.log(py[k]) + model[k].score(xx[np.newaxis, :])
    if np.argmax(score) == Y[n]:
        ncorrect1 += 1

    # 方法2: 最短距離法で識別
    d2 = np.sum(np.square(xx[np.newaxis, :] - xm), axis=1)
    if np.argmin(d2) == Y[n]:
        ncorrect2 += 1

print(f'方法1の識別率: {ncorrect1}/{N} = {ncorrect1/N:.3f}')
print(f'方法2の識別率: {ncorrect2}/{N} = {ncorrect2/N:.3f}')

識別率が逆転しました．方法1では $p(\mathbf{x}|y)$ に任意の正規分布を当てはめることができますが，そのせいで過適合が起きてしまったようです．

この授業で説明している範囲のあれこれでもう少し改良できますが，省略します（当てはめる正規分布に少し制約条件を付ける，次元削減を適用する，等）．

---
## 正規分布モデルではうまくいかない例
---

「密度推定」や「統計的識別入門」の notebook に出てくる2次元データの例は，正規分布の当てはめでうまくいく例ばかり見せていました．
うまくいかない例を観察するのも大事ですから，ここでそういう例を紹介しておきます．


まずはデータの準備．

In [ ]:
from scipy.stats import multivariate_normal
from sklearn.datasets import make_moons

moonX, moonY = make_moons(n_samples=(800, 200), noise=0.25)
NL, NT = 500, 500
XL, YL = moonX[:NL, :], moonY[:NL]
XT, YT = moonX[NL:, :], moonY[NL:]
print(XL.shape, YL.shape)
print(XT.shape, YT.shape)

In [ ]:
fig = plt.figure(figsize=(9, 6))
ax0 = fig.add_subplot(121)
ax0.scatter(XL[YL==0, 0], XL[YL==0, 1], marker='.', label='Class0')
ax0.scatter(XL[YL==1, 0], XL[YL==1, 1], marker='.', label='Class1')
ax0.set_xlim(-1.5, 2.5)
ax0.set_ylim(-2, 2)
ax0.set_aspect('equal')
ax0.legend()
ax0.set_title('training data')


ax1 = fig.add_subplot(122)
ax1.scatter(XT[YT==0, 0], XT[YT==0, 1], marker='.', label='Class0')
ax1.scatter(XT[YT==1, 0], XT[YT==1, 1], marker='.', label='Class1')
ax1.set_xlim(-1.5, 2.5)
ax1.set_ylim(-2, 2)
ax1.set_aspect('equal')
ax1.legend()
ax1.set_title('test data')

plt.show()

2クラス識別の例です．左が学習データ，右がテストデータ．

次のセルでは，学習データから事前確率 $p(y)$ とクラスごとの分布 $p(\mathbf{x}|y)$ を推定し，データをその事後確率 $p(y|\mathbf{x})$ （に比例する $p(\mathbf{x}|y)p(y)$ の値）が最大となるクラスに識別する実験が行なえます．

In [ ]:
K = 2
N, D = XL.shape

# 事前確率を推定
P = np.empty(K)
for k in range(K):
    P[k] = np.sum(YL==k)/N
print('P =', P)

# クラスごとのデータの分布を多変量正規分布でモデル化
mu = np.empty((K, D))
cov = np.empty((K, D, D))
model = np.empty(K, dtype=object)
for k in range(K):
    XX = XL[YL==k, :]
    model[k] = GaussianMixture(n_components=1, covariance_type='full')
    model[k].fit(XX)
    mu[k, ::] = model[k].means_
    cov[k, ::] = model[k].covariances_

# 学習データの識別
N = XL.shape[0]
LL = np.empty((N, K))
for n in range(N):
    for k in range(K):
        # log(p(y)p(x|y)) = log(p(y)) + log(p(x|y))
        LL[n, k] = np.log(P[k]) + model[k].score(XL[n, np.newaxis, :])
Yp = np.argmax(LL, axis=1)
ncorrect = np.sum(YL == Yp)
print(f'学習データの識別率: {ncorrect/N}')

# テストデータの識別
N = XT.shape[0]
LL = np.empty((N, K))
for n in range(N):
    for k in range(K):
        # log(p(y)p(x|y)) = log(p(y)) + log(p(x|y))
        LL[n, k] = np.log(P[k]) + model[k].score(XT[n, np.newaxis, :])
Yp = np.argmax(LL, axis=1)
ncorrect = np.sum(YT == Yp)
print(f'テストデータの識別率: {ncorrect/N}')

識別率の数値だけではうまくいってるのかどうかわかりませんので，可視化してみましょう．

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))
print(X_mesh.shape)

# 事後確率の推定
p = np.empty((K, X_mesh.shape[0]*X_mesh.shape[1]))
for k in range(K):
    proba = np.exp(model[k].score_samples(X_mesh.reshape((-1, 2))))
    p[k, :] = P[k] * proba
p /= np.sum(p, axis=0)
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(12, 8))

# 左図
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(XL[YL==k, 0], XL[YL==k, 1])
    z = multivariate_normal.pdf(X_mesh, mean=mu[k], cov=cov[k])
    ax0.contour(x_mesh, y_mesh, z, levels=4)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 右図
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(XL[YL==k, 0], XL[YL==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

左図は2クラスの学習データと，それに当てはめられた正規分布を可視化したものです．2クラスそれぞれの $p(\mathbf{x}|y)$ に単一の正規分布を当てはめるのでは，このデータのもつ複雑な分布を表しきれていないことは明らかですね．
右図は，事後確率の値を可視化したものです．2クラスをうまく分けられていないことがわかります（※）．

<span style="font-size: 75%">
※注: 「多変量解析及び演習」の「判別分析」の回に，正規分布を当てはめる場合には識別境界が2次曲線になることを説明していました．
</span>

---
## 混合正規分布モデルを使ってみる
---

※ ここで紹介する「混合正規分布」については，授業の範囲ではありません．式の説明は，眺めるだけでok．

上記で見たように，正規分布は単純すぎてうまくデータの分布を表現できないことがあります．
そういう場合への対応の方法はいろいろありますが，ここでは，「複数の正規分布の重み付き和」で表される「混合正規分布モデル」(Gaussian mixture model)を紹介します．

これは，$M$個の正規分布の重み付き和で，次のように表されます．
$$
p(\mathbf{x}) = \sum_{m=1}^{M} w_m{\cal N}(\mathbf{x}|\mathbf{\mu}_m, \Sigma_m)
$$
${\cal N}(\mathbf{x}|\mathbf{\mu}_m, \Sigma_m)$ は平均$\mathbf{\mu}_m$ 分散共分散行列 $\Sigma_m$ の多変量正規分布です．$w_m$ は $m$ 番目の正規分布の重みを表し，$0 \leq w_m \leq 1, \sum_{m=1}^M w_m = 1$ を満たします．
混合正規分布モデルでは，$w_m, \mathbf{\mu}_m, \Sigma_m$ がモデルパラメータとなります．

2次元の「うまくいかない例」のデータに $M=3$ の混合正規分布モデルを当てはめてみると，こんなふうになります．

In [ ]:
K = 2
N, D = XL.shape
M = 3

# 事前確率を推定
py = np.empty(K)
for k in range(K):
    py[k] = np.sum(YL==k)/N
print('### 事前確率')
print('p(y) = ', py)

# p(x|y) をGMMでモデル化
mu = np.empty((K, M, D))
cov = np.empty((K, M, D, D))
model = np.empty(K, dtype=object)
for k in range(K):
    XX = XL[YL==k, :]
    model[k] = GaussianMixture(n_components=M, covariance_type='full')
    model[k].fit(XX)
    mu[k, ::] = model[k].means_
    cov[k, ::] = model[k].covariances_

# 学習データの識別
N = XL.shape[0]
LL = np.empty((N, K))
for n in range(N):
    for k in range(K):
        # log(p(y)p(x|y)) = log(p(y)) + log(p(x|y))
        LL[n, k] = np.log(P[k]) + model[k].score(XL[n, np.newaxis, :])
Yp = np.argmax(LL, axis=1)
ncorrect = np.sum(YL == Yp)
print(f'学習データの識別率: {ncorrect/N}')

# テストデータの識別
N = XT.shape[0]
LL = np.empty((N, K))
for n in range(N):
    for k in range(K):
        # log(p(y)p(x|y)) = log(p(y)) + log(p(x|y))
        LL[n, k] = np.log(P[k]) + model[k].score(XT[n, np.newaxis, :])
Yp = np.argmax(LL, axis=1)
ncorrect = np.sum(YT == Yp)
print(f'テストデータの識別率: {ncorrect/N}')

学習データテストデータともに，単一の正規分布を当てはめたときよりもずっと高い識別率が得られています．

クラスごとの分布と事後確率の推定値を可視化してみると，こんなん．

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))
print(X_mesh.shape)

# 事後確率の推定
p = np.empty((K, X_mesh.shape[0]*X_mesh.shape[1]))
for k in range(K):
    proba = np.exp(model[k].score_samples(X_mesh.reshape((-1, 2))))
    p[k, :] = py[k] * proba
p /= np.sum(p, axis=0)
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(12, 8))

# Gaussian を当てはめた結果
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(XL[YL==k, 0], XL[YL==k, 1])
    for m in range(M):
        z = multivariate_normal.pdf(X_mesh, mean=mu[k, m], cov=cov[k, m])
        ax0.contour(x_mesh, y_mesh, z, levels=4)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 事後確率の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(XL[YL==k, 0], XL[YL==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

`M = 3` のところの数を変えると，正規分布の数を変えられます．いろいろ変えてみるとよいでしょう（あまり多くするとエラーになるかもしれません）．

手書き数字の例に混合正規分布モデルを使ってみるのも面白いのですが，省略します．

---
## 識別モデルいろいろ
---

この notebook のここまでは，ex14notebookA「統計的識別入門」で説明した方法，すなわち，事後確率 $p(y|\mathbf{x})$ のかわりに事前確率 $p(y)$ とクラスごとのデータ分布 $p(\mathbf{x}|y)$ を推定し $p(\mathbf{x}|y)p(y)$ の値によって識別をするという方法の実験でした．このようにクラスごとのデータ分布 $p(\mathbf{x}|y)$ の推定を経由して識別する方法は，生成モデルによる方法と呼ばれることがあります．

一方，以前登場したロジスティック回帰やニューラルネットを用いる方法は，クラスごとのデータ分布の推定を経由せず，事後確率を直接モデル化する方法といえます．そのような方法は，生成モデルと対比させて，識別モデルによる方法と呼ばれることがあります．

以下では，いくつかの代表的な識別モデルを用いて，上と同じデータの識別をさせてみましょう．

----
### ロジスティック回帰


これまでも何度も話題に上がり，「機械学習・人工知能の歴史」でも触れたように，ロジスティック回帰は，線形分離可能なデータしかうまく識別できません．そのことを再確認しましょう．

以下のコードセルを実行し，識別率や識別境界の形を確認しましょう．

In [ ]:
# モデルの作成
model = LogisticRegression(C=1.0, class_weight='balanced')

# 学習
model.fit(XL, YL)

In [ ]:
# 識別率の算出
print('学習データの識別率:', model.score(XL, YL))
print('テストデータの識別率:', model.score(XT, YT))

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))
print(X_mesh.shape)

# 事後確率の推定
p = model.predict_proba(X_mesh.reshape((-1, 2))).T
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(6, 6))

# 事後確率の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(111)
for k in range(K):
    ax1.scatter(XL[YL==k, 0], XL[YL==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

----
### Support Vector Machine 使ってみる


「歴史」の話の中で登場した SVM (Support Vector Machine)です．

SVMには様々なハイパーパラメータがあり，実際にはクロスバリデーション等の方法でそれらを決定するべきなのですが，ここでは決め打ちで実験することにします．興味のある人向けに補足しておくと，この実験では，RBF(Radial Basis Function)カーネルを用い，正則化パラメータ 'C' の値のみ何通りか変えられるようにしています．

In [ ]:
# モデルの作成
model = SVC(C=1.0, kernel='rbf', probability=True, class_weight='balanced')

# 学習
model.fit(XL, YL)

In [ ]:
# 識別率の算出
print('学習データの識別率:', model.score(XL, YL))
print('テストデータの識別率:', model.score(XT, YT))

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))
print(X_mesh.shape)

# 事後確率の推定
p = model.predict_proba(X_mesh.reshape((-1, 2))).T
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(6, 6))

# 事後確率の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(111)
for k in range(K):
    ax1.scatter(XL[YL==k, 0], XL[YL==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

一度動かしたら，次の `C` の値（正則化パラメータ）を 0.01 や 100.0 等に変えて実行してみるとよいでしょう．

```
# モデルの作成
model = SVC(C=1.0, kernel='rbf', probability=True, class_weight='balanced')
```

----
### ニューラルネット使ってみる

階層型ニューラルネット（多層パーセプトロン）使ってみましょう．

階層型ニューラルネットの場合，ネットワークに中間層をいくつ持たせるか，それぞれの中間層のニューロン数をいくつにするか，活性化関数にどんなものを用いるか，等々のハイパーパラメータがありますが，ここでは一通りに決め打ちで実験します．

In [ ]:
# モデルの作成
model = MLPClassifier(hidden_layer_sizes=[1000, 1000], activation='relu', verbose=True)

# 学習
model.fit(XL, YL)

In [ ]:
# 識別率の算出
print('学習データの識別率:', model.score(XL, YL))
print('テストデータの識別率:', model.score(XT, YT))

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))
print(X_mesh.shape)

# 事後確率の推定
p = model.predict_proba(X_mesh.reshape((-1, 2))).T
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(6, 6))

# 事後確率の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(111)
for k in range(K):
    ax1.scatter(XL[YL==k, 0], XL[YL==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()